In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

import matplotlib as mpl
import numpy as np
import pandas as pd

from rcpl.config import BASE_DIR, MPL_RC, CONFIGS_DIR
from rcpl.utils.prediction import ChainPredictor, format_float

mpl.rcParams.update(MPL_RC)
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [16]:
exp_path, e_id = BASE_DIR / 'data/epsp_stress/measured/2023-11-23.json', 1

In [17]:
gru_predictor = ChainPredictor(config_path=CONFIGS_DIR / 'maftr/maftr2_4.best/gru.yaml', predicts_scaled_theta=True)

In [18]:
unscaled_theta_opt_gru, stress_pred_opt_gru, _ = gru_predictor.predict_from_json_simplex(exp_path, verbose=True)

Origin score: 812.12, Nelder-Mead score: 398.05


In [19]:
fraction = 1
validate_crlb_opt = gru_predictor.chain.validate_crlb.force().value(exp_path, crop_signal=[0, 681], unscaled_theta=unscaled_theta_opt_gru, fraction=fraction)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 259.18it/s]


In [20]:
unscaled_theta_prediction_dict = {f'${key}$': val for key, val in zip(gru_predictor.chain.dataset_info.value['latex_labels'], unscaled_theta_opt_gru)}

In [21]:
df_params = pd.DataFrame(unscaled_theta_prediction_dict.items(), columns=['parameter', r'$\widehat{\boldsymbol\theta}$'])
df_params['ESTD'] = [i / fraction for i in validate_crlb_opt['theta_std']]
df_params[r'$\sqrt{\mathrm{CRLB}}$'] = [np.sqrt(i) for i in validate_crlb_opt['crlb']['std']]
df_params[r'ESTD/$\widehat{\boldsymbol\theta}$'] = df_params['ESTD'] / df_params[r'$\widehat{\boldsymbol\theta}$']
df_params

,parameter,$\widehat{\boldsymbol\theta}$,ESTD,$\sqrt{\mathrm{CRLB}}$,asdf$
0,$k_0$,195.324219,2.534019,0.793665,0.012973
1,$\kappa_1$,23206.996094,190.536682,16.781239,0.008210
2,$\kappa_2$,0.028727,0.000891,0.016827,0.031032
3,$c_1$,31843.533203,360.462860,16.085636,0.011320
4,$c_2$,2539.827637,142.136322,3.221019,0.055963
5,$c_3$,424.162659,62.868717,5.868053,0.148218
6,$c_4$,410.384888,108.958427,7.932480,0.265503
7,$a_1$,67.578262,6.086324,0.532633,0.090063
8,$a_2$,138.944458,5.721293,0.635240,0.041177
9,$a_3$,16.279949,8.293461,0.854660,0.509428
